# 작업 노동자 데이터 기반 사고 확률 예측

- **공정율 (`progress_rate`)**: 공사의 진행 정도를 나타내는 10% 단위의 비율 (10%, 20%, ..., 90%).
- **공사비 (`construction_cost`)**: 해당 공사의 총 공사비용 (억 단위).
- **공종 (`construction_type`)**: 공사의 종류 (1: 토목/기초, 2: 골조/구조, 3: 설비/마감, 4: 외장/조경).
- **근무기간 (`work_duration`)**: 노동자의 현재 근무지에서의 근무 기간 (월 단위).
- **외국인 근무자 여부 (`foreign_worker`)**: 노동자가 외국인인지 여부 (1: 외국인, 0: 내국인).
- **연령 (`age`)**: 노동자의 나이.
- **교육빈도/교육이수횟수 (`education_count`)**: 노동자가 안전 교육을 받은 총 횟수 (Potential Accident Level 기반).
- **사고 이력 (`accident_history`)**: 과거에 사고를 경험한 적이 있는지 여부 (1: 있음, 0: 없음).
- **안전장구 미착용 경고 횟수 (`warnings`)**: 안전 장비 미착용으로 받은 경고 횟수.
- **작업자 숙련도 (`skill_level`)**: 노동자의 숙련도 (1: 하, 2: 중, 3: 상).
- **안전 규정 준수 점수 (`compliance_score`)**: 안전 규정을 준수하는 정도를 평가한 점수 (0~100점, Potential Accident Level 기반).
- **작업 시간 (`work_hours`)**: 일일 평균 작업 시간 (시간 단위).
- **사고 여부 (`accident`)**: 해당 작업의 사고 여부 (1: 사고, 0: 사고 없음).

데이터 원본: https://www.kaggle.com/datasets/ihmstefanini/industrial-safety-and-health-analytics-database

In [25]:
# 1. 패키지 불러오기 및 데이터 로드
import pandas as pd
import sweetviz as sv
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, precision_score, recall_score, accuracy_score, classification_report
import matplotlib.pyplot as plt
import seaborn as sns

# 데이터 로드
data = pd.read_excel('2조_데이터.xlsx')

In [ ]:
# 2. 데이터 구조 보기
print("데이터 구조:")
print(data.info())
print("\n데이터 샘플:")
print(data.head())

In [ ]:
# 3. Sweetviz를 활용한 EDA
eda_report = sv.analyze(data)
eda_report.show_notebook()

In [26]:
# 4. 데이터 분리
X = data.drop(columns=['accident'])  # 독립 변수
y = data['accident']                 # 종속 변수
# 수치형 변수만 선택 (Random Forest는 숫자 데이터를 요구)
X = pd.get_dummies(X, columns=['construction_type', 'foreign_worker', 'skill_level'], drop_first=True)

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

In [27]:
# 5. Random Forest 모델링
model = RandomForestClassifier(random_state=42, n_estimators=100)
model.fit(X_train, y_train)

# 예측
y_pred = model.predict(X_test)

In [ ]:
# 6. 결과 평가
conf_matrix = confusion_matrix(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
accuracy = accuracy_score(y_test, y_pred)

print("Confusion Matrix:")
print(conf_matrix)
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

print(f"Precision: {precision:.2f}, Recall: {recall:.2f}, Accuracy: {accuracy:.2f}")

In [ ]:
# 7. Feature Importance 시각화
importances = model.feature_importances_
features = X.columns

plt.figure(figsize=(10, 6))
sns.barplot(x=importances, y=features)
plt.title('Feature Importance')
plt.xlabel('Importance Score')
plt.ylabel('Feature')
plt.show()